In [2]:
import pandas as pd
import os
os.chdir("..")
os.getcwd()
order_df = pd.read_csv('/home/martin/Desktop/10-Academy/week-8/Causal-Inference/cleaned_completed_orders.csv')
requests_df = pd.read_csv('/home/martin/Desktop/10-Academy/week-8/Causal-Inference/cleaned_delivery_requests.csv')


In [6]:
print(order_df.columns)
print(requests_df.columns)


Index(['Trip ID', 'Trip Origin', 'Trip Destination', 'Trip Start Time',
       'Trip End Time', 'Trip Duration', 'Trip Start Hour', 'Trip Start Day',
       'Trip Start Month', 'Trip Start Weekday', 'Is Weekend', 'Weather',
       'Is Holiday', 'Origin Lat', 'Origin Lon', 'Destination Lat',
       'Destination Lon', 'Trip Distance', 'Speed'],
      dtype='object')
Index(['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng'], dtype='object')


In [8]:
merged_df = pd.merge(order_df, requests_df, left_on='Trip ID', right_on='order_id')


In [13]:
merged_df.columns

Index(['Trip ID', 'Trip Origin', 'Trip Destination', 'Trip Start Time',
       'Trip End Time', 'Trip Duration', 'Trip Start Hour', 'Trip Start Day',
       'Trip Start Month', 'Trip Start Weekday', 'Is Weekend', 'Weather',
       'Is Holiday', 'Origin Lat', 'Origin Lon', 'Destination Lat',
       'Destination Lon', 'Trip Distance', 'Speed', 'id', 'order_id',
       'driver_id', 'driver_action', 'lat', 'lng'],
      dtype='object')

In [15]:
# Group the merged dataframe by order_id and calculate the trip information
grouped_df = merged_df.groupby('order_id').agg({
    'Trip Origin': 'first',
    'Trip Destination': 'first',
    'Trip Start Weekday': 'first',
    'Trip Start Day': 'first',
    'Is Holiday': 'first',
    'Weather': 'first',
    'Trip Distance': 'first',
    'Speed': 'first'
})

In [16]:
# Reset the index to get a column for order_id
grouped_df = grouped_df.reset_index()

In [20]:

# Rename the columns to match the original order.csv dataframe
grouped_df = grouped_df.rename(columns={
   'Trip Origin': 'Origin',
    'Trip Destination': 'Destination',
    'Trip Start Weekday': 'Weekday',
    'Trip Start Day': 'Trip_Start_Day',
    'Is Holiday': 'Holiday',
    'Weather': 'Rain',
    'Trip Distance': 'Distance',
    'Speed': 'Speed'
})


In [22]:
result_df = pd.merge(requests_df, grouped_df, on='order_id', how='left')
result_df.tail(5)

,id,order_id,driver_id,driver_action,lat,lng,Trip_Origin,Trip_Destination,Weekday,Trip_Start_Day,Holiday,Rain,Distance,Speed
1557735,1557736,517948,243774,rejected,6.469036,3.566877,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",4.0,10.0,False,No-Rain,8.136366,0.000366
1557736,1557737,517948,245447,rejected,6.425431,3.527358,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",4.0,10.0,False,No-Rain,8.136366,0.000366
1557737,1557738,517948,239866,rejected,6.440013,3.525378,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",4.0,10.0,False,No-Rain,8.136366,0.000366
1557738,1557739,517948,243774,rejected,6.469036,3.566877,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",4.0,10.0,False,No-Rain,8.136366,0.000366
1557739,1557740,517948,244170,rejected,6.469390,3.572079,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",4.0,10.0,False,No-Rain,8.136366,0.000366


In [23]:
# Save the resulting dataframe to a new CSV file to inspect
result_df.to_csv('merged.csv', index=False)

In [25]:
requests_df = requests_df[['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng']]


In [26]:
# Concatenate the original order.csv dataframe with the new columns
requests_df = pd.concat([requests_df[requests_df.columns[:6]], grouped_df], axis=1)
requests_df.tail()

,id,order_id,driver_id,driver_action,lat,lng,order_id,Trip_Origin,Trip_Destination,Weekday,Trip_Start_Day,Holiday,Rain,Distance,Speed
1557735,1557736,517948,243774,rejected,6.469036,3.566877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557736,1557737,517948,245447,rejected,6.425431,3.527358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557737,1557738,517948,239866,rejected,6.440013,3.525378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557738,1557739,517948,243774,rejected,6.469036,3.566877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1557739,1557740,517948,244170,rejected,6.469390,3.572079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
print(requests_df.columns)


Index(['id', 'order_id', 'driver_id', 'driver_action', 'lat', 'lng',
       'order_id', 'Trip_Origin', 'Trip_Destination', 'Weekday',
       'Trip_Start_Day', 'Holiday', 'Rain', 'Distance', 'Speed'],
      dtype='object')


In [28]:
print(grouped_df.columns)

Index(['order_id', 'Trip_Origin', 'Trip_Destination', 'Weekday',
       'Trip_Start_Day', 'Holiday', 'Rain', 'Distance', 'Speed'],
      dtype='object')


In [29]:

# Save the updated dataframe to a new CSV file
order_df.to_csv('final.csv', index=False)
